In [4]:
!wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

1.sh  bank-full.csv  bank+marketing.zip  Untitled.ipynb


In [5]:
!unzip bank+marketing.zip

Archive:  bank+marketing.zip
 extracting: bank.zip                
 extracting: bank-additional.zip     


In [7]:
!unzip bank.zip

Archive:  bank.zip
  inflating: bank-full.csv           
  inflating: bank-names.txt          
  inflating: bank.csv                


In [8]:
import pandas as pd

In [13]:
df = pd.read_csv("bank-full.csv",delimiter=";")

In [15]:
df.head().T

,0,1,2,3,4
age,58,44,33,47,33
job,management,technician,entrepreneur,blue-collar,unknown
marital,married,single,married,married,single
education,tertiary,secondary,secondary,unknown,unknown
default,no,no,no,no,no
balance,2143,29,2,1506,1
housing,yes,yes,yes,yes,no
loan,no,no,yes,no,no
contact,unknown,unknown,unknown,unknown,unknown
day,5,5,5,5,5


In [16]:
columns_to_keep = [
    'age', 'job', 'marital', 'education', 'balance', 
    'housing', 'contact', 'day', 'month', 'duration', 
    'campaign', 'pdays', 'previous', 'poutcome', 'y'
]
df = df[columns_to_keep]

In [17]:
df.isna().any()

age          False
job          False
marital      False
education    False
balance      False
housing      False
contact      False
day          False
month        False
duration     False
campaign     False
pdays        False
previous     False
poutcome     False
y            False
dtype: bool

In [18]:
df.education.mode()

0    secondary
Name: education, dtype: object

In [20]:
numerical_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']


In [21]:
# Compute the correlation matrix for the numerical columns
correlation_matrix = df[numerical_columns].corr()

# Display the correlation matrix
print(correlation_matrix)

# Find the pair of features with the highest correlation (ignoring self-correlations)
correlation_pairs = correlation_matrix.unstack()
# Remove self-correlations (i.e., 1.0 diagonal values)
correlation_pairs = correlation_pairs[correlation_pairs != 1.0].sort_values(ascending=False)

# Display the pair with the highest correlation
print("The two features with the biggest correlation are:")
print(correlation_pairs.head(1))

               age   balance       day  duration  campaign     pdays  previous
age       1.000000  0.097783 -0.009120 -0.004648  0.004760 -0.023758  0.001288
balance   0.097783  1.000000  0.004503  0.021560 -0.014578  0.003435  0.016674
day      -0.009120  0.004503  1.000000 -0.030206  0.162490 -0.093044 -0.051710
duration -0.004648  0.021560 -0.030206  1.000000 -0.084570 -0.001565  0.001203
campaign  0.004760 -0.014578  0.162490 -0.084570  1.000000 -0.088628 -0.032855
pdays    -0.023758  0.003435 -0.093044 -0.001565 -0.088628  1.000000  0.454820
previous  0.001288  0.016674 -0.051710  0.001203 -0.032855  0.454820  1.000000
The two features with the biggest correlation are:
previous  pdays    0.45482
dtype: float64


In [24]:
# Target encodign
df.y = df.y.map(dict(yes=1, no=0))

In [25]:
df.head().T

,0,1,2,3,4
age,58,44,33,47,33
job,management,technician,entrepreneur,blue-collar,unknown
marital,married,single,married,married,single
education,tertiary,secondary,secondary,unknown,unknown
balance,2143,29,2,1506,1
housing,yes,yes,yes,yes,no
contact,unknown,unknown,unknown,unknown,unknown
day,5,5,5,5,5
month,may,may,may,may,may
duration,261,151,76,92,198


In [27]:
from sklearn.model_selection import train_test_split
# Separate the target variable 'y' from the rest of the DataFrame
X = df.drop('y', axis=1)
y = df['y']

# First, split the data into train (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

# Now, split the temp data into validation (20%) and test (20%) sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Verify the size of each dataset
print(f'Train set size: {X_train.shape}')
print(f'Validation set size: {X_val.shape}')
print(f'Test set size: {X_test.shape}')from sklearn.feature_selection import mutual_info_classif

Train set size: (27126, 14)
Validation set size: (9042, 14)
Test set size: (9043, 14)


In [30]:
from sklearn.feature_selection import mutual_info_classif
categorical_columns = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome'] 
X_train_categorical = X_train[categorical_columns]



In [34]:
from sklearn.preprocessing import LabelEncoder
# Label encode the categorical columns
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X_train_categorical[col] = le.fit_transform(X_train_categorical[col])
    label_encoders[col] = le  # Save the encoder in case you need it later

# Calculate the mutual information scores between y_train and the categorical variables
mi_scores = mutual_info_classif(X_train_categorical, y_train, discrete_features=True)


/tmp/ipykernel_1710778/2765687987.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_categorical[col] = le.fit_transform(X_train_categorical[col])
/tmp/ipykernel_1710778/2765687987.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_categorical[col] = le.fit_transform(X_train_categorical[col])
/tmp/ipykernel_1710778/2765687987.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [35]:
mi_scores_rounded = [round(score, 2) for score in mi_scores]
mi_scores_df = pd.DataFrame({
    'Categorical Variable': categorical_columns,
    'Mutual Information Score': mi_scores_rounded
})
mi_scores_df

,Categorical Variable,Mutual Information Score
0,job,0.01
1,marital,0.00
2,education,0.00
3,housing,0.01
4,contact,0.01
5,month,0.02
6,poutcome,0.03


In [37]:

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

# Define the one-hot encoder for categorical columns
ohe = OneHotEncoder(drop='first')

# Define a column transformer that applies the one-hot encoder only to categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', ohe, categorical_columns)
    ], remainder='passthrough'  # Leave other columns unchanged
)

# Define the logistic regression model with specified parameters
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

# Create a pipeline that first preprocesses the data (one-hot encoding) and then applies logistic regression
pipeline = make_pipeline(preprocessor, model)

# Fit the model to the training data
pipeline.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = pipeline.predict(X_val)

# Calculate the accuracy on the validation set
accuracy = accuracy_score(y_val, y_val_pred)

# Round the accuracy to 2 decimal places
accuracy_rounded = round(accuracy, 2)

# Display the accuracy
print(f'Validation Accuracy: {accuracy_rounded}')

Validation Accuracy: 0.9


In [41]:
X_val.head().T

,31415,5407,34105,25943,2053
age,36,25,35,56,45
job,management,blue-collar,management,self-employed,blue-collar
marital,married,single,single,married,married
education,secondary,primary,tertiary,tertiary,primary
balance,565,-221,1162,0,101
housing,no,yes,no,no,yes
contact,cellular,unknown,cellular,cellular,unknown
day,27,23,30,19,12
month,mar,may,apr,nov,may
duration,300,250,175,122,460


In [52]:
all_columns = categorical_columns + ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

baseline_accuracy = accuracy_score(y_val, y_val_pred)

# Store the differences when each feature is removed
feature_differences = {}

# Loop through all columns and remove each one, retrain the model, and record accuracy
for feature in all_columns:

    # Drop the feature from both categorical and numerical columns lists
    if feature in categorical_columns:
        current_categorical_columns = [col for col in categorical_columns if col != feature]
        current_numerical_columns = numerical_columns
    else:
        current_categorical_columns = categorical_columns
        current_numerical_columns = [col for col in numerical_columns if col != feature]
    
    # Create a new preprocessor that accounts for the remaining columns
    preprocessor_temp = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(drop='first'), current_categorical_columns)
        ], remainder='passthrough'  # Leave the remaining numerical columns unchanged
    )

    
    # Drop the feature from the training and validation set
    X_train_temp = X_train.drop(feature, axis=1)
    X_val_temp = X_val.drop(feature, axis=1)
    
    # Create a new pipeline and fit without the feature
    pipeline_temp = make_pipeline(preprocessor_temp, model)
    #print(X_train_temp.head().T)
    #print(y_train.head().T)
    
    pipeline_temp.fit(X_train_temp, y_train)
    
    # Predict on the validation set without the feature
    y_val_pred_temp = pipeline_temp.predict(X_val_temp)
    
    # Calculate the accuracy without the feature
    accuracy_without_feature = accuracy_score(y_val, y_val_pred_temp)

    print(f"accuracy without {feature}: {accuracy_without_feature}")
    # Calculate the difference in accuracy
    accuracy_difference = baseline_accuracy - accuracy_without_feature
    
    # Store the difference for this feature
    feature_differences[feature] = accuracy_difference

# Find the feature with the smallest difference
least_useful_feature = min(feature_differences, key=feature_differences.get)

# Display the differences
print("Feature Differences:", feature_differences)
print("Least Useful Feature:", least_useful_feature)

accuracy without job: 0.8996903339969033
accuracy without marital: 0.9001327140013271
accuracy without education: 0.9001327140013271
accuracy without housing: 0.9012386640123866
accuracy without contact: 0.8998009289980093
accuracy without month: 0.9024552090245521
accuracy without poutcome: 0.8913957089139571
accuracy without age: 0.9009068790090687
accuracy without balance: 0.9002433090024331
accuracy without day: 0.900575094005751
accuracy without duration: 0.893718203937182
accuracy without campaign: 0.900464499004645
accuracy without pdays: 0.900353904003539
accuracy without previous: 0.9006856890068569
Feature Differences: {'job': 0.0008847600088476293, 'marital': 0.00044238000442387015, 'education': 0.00044238000442387015, 'housing': -0.0006635700066356387, 'contact': 0.000774165007741634, 'month': -0.0018801150188011428, 'poutcome': 0.009179385091793835, 'age': -0.00033178500331776384, 'balance': 0.00033178500331787486, 'day': 0.0, 'duration': 0.006856890068568933, 'campaign': 

In [50]:
feature_differences["age"]

-0.00033178500331776384

In [51]:
feature_differences["balance"]

0.00033178500331787486

In [55]:
feature_differences["marital"]

0.00044238000442387015

In [53]:
feature_differences["previous"]

-0.00011059500110588427

In [ ]:
# Q6
# Define the logistic regression model parameters to try
C_values = [0.01, 0.1, 1, 10, 100]

# Define categorical columns
categorical_columns = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']
numerical_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# One-hot encode categorical columns
ohe = OneHotEncoder(drop='first')
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', ohe, categorical_columns)
    ], remainder='passthrough'  # Leave the numerical columns unchanged
)

# Store the accuracies for each C
accuracies = []

# Loop over the different values of C
for C in C_values:
    # Define the logistic regression model with the current C
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    
    # Create the pipeline (preprocessing + model)
    pipeline = make_pipeline(preprocessor, model)
    
    # Fit the model on the training data
    pipeline.fit(X_train, y_train)
    
    # Predict on the validation set
    y_val_pred = pipeline.predict(X_val)
    
    # Calculate the accuracy on the validation set
    accuracy = accuracy_score(y_val, y_val_pred)
    
    # Round the accuracy to 3 decimal places and store it
    accuracies.append(round(accuracy, 3))

# Find the best C (the one with the highest accuracy)
best_accuracy = max(accuracies)
best_C = C_values[accuracies.index(best_accuracy)]

# Display the results
print(f'Accuracies for different C values: {accuracies}')
print(f'The best C value is: {best_C}')